# CUP IT FINAL Секция Data Science

## Команда Iguana Team

### Этот ноутбук содержит код:

- Загрузки и предобработки датасета
- Объявления и обучения модели

# Импортируем библиотеки

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import copy
from utils import get_tags, compute_metrics, predict_and_print_metrics, MultiNLI_dataset, preprocess_tokenize_dataset
from modeling import SemBERT, train_model

# Загружаем и инициализируем датасет

In [ ]:
!wget -q https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip
!unzip -q ./multinli_1.0.zip

In [ ]:
data_folder = "./multinli_1.0/"

In [ ]:
df_train = pd.read_json(f"{data_folder}multinli_1.0_train.jsonl", lines=True)
## Остальные датасеты отложим на потом, чтобы не загружать оперативную память

# Выделяем тэги с помощью регулярных выражений

In [ ]:
df_train['sentence1_tagged'] = df_train['sentence1_parse'].apply(get_tags)
df_train['sentence2_tagged'] = df_train['sentence2_parse'].apply(get_tags)

In [ ]:
## Оставляем только предложения, тэги и закодированный label, для оптимизации удаляем очень длинные строки
label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
df = df_train[df_train['gold_label'] != '-']
df['gold_label'].replace(to_replace=label_set, inplace=True)
df = df[['sentence1', 'sentence1_tagged', 'sentence2', 'sentence2_tagged', 'gold_label']].loc[df[(df['sentence1'].str.len() < 250) & (df['sentence2'].str.len() < 250)].index]

In [ ]:
punctuation = '!"#$%&\'()*,-./:;<=>?@^_`{|}~'

# Приводим к нижнему регистру, убираем пунктуацию

sentences1 = list(df['sentence1'].str.lower().replace(re.compile(punctuation), ' ').values)
sentences2 = list(df['sentence2'].str.lower().replace(re.compile(punctuation), ' ').values)
sentences1_tags = list(df['sentence1_tagged'].values)
sentences2_tags = list(df['sentence2_tagged'].values)
labels = df['gold_label'].values

# Инициализируем токенайзер и создаем датасет

In [ ]:
from transformers import DistilBertTokenizerFast, BertTokenizerFast
import torch
import torch.nn as nn
import transformers
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda:0')
torch.cuda.get_device_name()

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# Токенизируем и возвращаем в тензорах с пэддингом до самой длинной последовательности и освобождаем память
# Пэддинг нужен для того, чтобы собирались батчи, но увеличение длины последовательности очень сильно прибавляет времени тренировки. 
# Токенизатор возвращает свой класс, нужны только input_ids и attention_mask

tokenized_pairs = tokenizer(text=sentences1, text_pair=sentences2, padding=True, return_tensors='pt')
del df_train, sentences1, sentences2

# Создаем токенайзер из словаря тэгов

In [ ]:
tags_set = set()
for sent in df['sentence1_tagged'].values:
    tags_set = tags_set | set(sent.split(' '))

In [ ]:
tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
with open('./vocab.txt', 'w') as file:
    file.write('\n'.join(tokens) + '\n')
    file.write('\n'.join(list(tags_set)) + '\n')

In [ ]:
tag_tokenizer = DistilBertTokenizerFast(vocab_file='./vocab.txt', do_lower_case=False)

# Выясняем максимальную длину токенизированных предложений из тэгов

Это нужно для того, чтобы во время тренировки токенайзер возвращал тензоры одной длины, для возможности собрать их в батчи

In [ ]:
max_len_train = 0
text = df['sentence1_tagged'].values.astype(str)
text_pairs = df['sentence2_tagged'].values.astype(str)

for seq1, seq2 in zip(text, text_pairs):
    input_ids = tag_tokenizer(seq1, text_pair=seq2, add_special_tokens=True)['input_ids']
    if max_len_train < len(input_ids):
        max_len_train = max(max_len_train, len(input_ids))

del text, text_pairs

In [ ]:
train_dataset = MultiNLI_dataset(tokenized_pairs, labels, sentences1_tags, sentences2_tags, max_len_train)
del tokenized_pairs

# Повторяем операции для валидационных датасетов

In [ ]:
df_matched = pd.read_json(f"{data_folder}multinli_1.0_dev_matched.jsonl", lines=True)
df_mismatched = pd.read_json(f"{data_folder}multinli_1.0_dev_mismatched.jsonl", lines=True)

In [ ]:
tokenized_pairs_mis, labels_mis, mis_tags_sentence_1, mis_tags_sentence_2, max_len_mis = 
    preprocess_tokenize_dataset(tokenizer, df_mismatched)

tokenized_pairs_mat, labels_mat, mat_tags_sentence_1, mat_tags_sentence_2, max_len_mat = 
    preprocess_tokenize_dataset(tokenizer, df_matched)

In [ ]:
mis_dataset = MultiNLI_dataset(tokenized_pairs_mis, labels_mis, mis_tags_sentence_1, mis_tags_sentence_2, max_len_mis)
mat_dataset = MultiNLI_dataset(tokenized_pairs_mat, labels_mat, mat_tags_sentence_1, mat_tags_sentence_2, max_len_mat)

# Инициализируем и тренируем модель

In [ ]:
num_epochs = 3
model = SemBERT()
model = model.to(device)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=80, shuffle=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0)
lr_scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=len(train_dataloader)*num_epochs)
loss_fct = torch.nn.CrossEntropyLoss()

In [ ]:
train_model(model, training_dataloader=train_dataloader, optimizer=optimizer, 
                                                          lr_scheduler=lr_scheduler, loss_fct=loss_fct, num_epochs=num_epochs)

In [ ]:
torch.cuda.empty_cache()

# Сохраняем модель

In [ ]:
torch.save(model, './model.pth')